# Unit 7 : AI vs AI - Multi Agent Reinforcement Learning(MARL)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit0/thumbnail.png" alt="Thumbnail"/>

**Full Tutorial**: https://huggingface.co/learn/deep-rl-course/unit7/hands-on

# What is AI vs. AI?

AI vs. AI is an open-source tool we developed at Hugging Face to compete agents on the Hub against one another in a multi-agent setting. These models are then ranked in a leaderboard.

The idea of this tool is to have a robust evaluation tool: **by evaluating your agent with a lot of others, you’ll get a good idea of the quality of your policy.**

More precisely, AI vs. AI is three tools:

* A **matchmaking process** defining the matches (which model against which) and running the model fights using a background task in the Space.
* A **leaderboard** getting the match history results and displaying the models’ ELO ratings: https://huggingface.co/spaces/huggingface-projects/AIvsAI-SoccerTwos
* A **Space demo** to visualize your agents playing against others: https://huggingface.co/spaces/unity/ML-Agents-SoccerTwos

In addition to these three tools, your classmate cyllum created a 🤗 SoccerTwos Challenge Analytics where you can check the detailed match results of a model: https://huggingface.co/spaces/cyllum/soccertwos-analytics

We’re wrote [a blog post to explain this AI vs. AI tool in detail](https://huggingface.co/blog/aivsai), but to give you the big picture it works this way:

* Every four hours, our algorithm fetches all the available models for a given environment (in our case ML-Agents-SoccerTwos).
* It creates a queue of matches with the matchmaking algorithm.
* We simulate the match in a Unity headless process and gather the match result (1 if the first model won, 0.5 if it’s a draw, 0 if the second model won) in a Dataset.
* Then, when all matches from the matches queue are done, we update the ELO score for each model and update the leaderboard.

## Competition Rules

This first AI vs. AI competition **is an experiment**: the goal is to improve the tool in the future with your feedback. So **some breakups can happen during the challenge**. But don’t worry all the results are saved in a dataset so we can always restart the calculation correctly without losing information.

In order for your model to get correctly evaluated against others you need to follow these rules:

* You can’t change the observation space or action space of the agent. By doing that your model will not work during evaluation.
* You can’t use a custom trainer for now, you need to use the Unity MLAgents ones.
* We provide executables to train your agents. You can also use the Unity Editor if you prefer , but to avoid bugs, we advise that you use our executables.

What will make the difference during this challenge are **the hyperparameters you choose**.

# Step 0: Install MLAgents and download the correct executable

## Conda Install

Fist install conda as we need to work in a conda environment. There maybe incompatibality with local env/colab/kaggle environemnt. After that, open the terminal in the directory where you will work with unity mlagents and type these commands. I am creating this notebook for organizing all commands with enough documentation. But you have to work on cmd prompt or some terminals and enter these commands accordingly

In [ ]:
!conda create --name rl python=3.10.12 -y

In [ ]:
# !conda init
!conda activate rl

In [ ]:
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

In [ ]:
%cd ml-agents
!pip install -e ./ml-agents-envs
!pip install -e ./ml-agents

Now that it’s installed, we need to add the environment training executable. Based on your operating system you need to download one of them, unzip it and place it in a new folder inside ml-agents that you call training-envs-executables

At the end your executable should be in ml-agents/training-envs-executables/SoccerTwos

Windows: Download this [executable](https://drive.google.com/file/d/1sqFxbEdTMubjVktnV4C6ICjp89wLhUcP/view)

# Step 1: Understand the environment

The environment is called SoccerTwos. The Unity MLAgents Team made it. You can find its documentation [here](https://github.com/Unity-Technologies/ml-agents/blob/develop/docs/Learning-Environment-Examples.md#soccer-twos)

The goal in this environment is to get the ball into the opponent’s goal while preventing the ball from entering your own goal.


⬇️ Here is an example of **2 vs 2 soccer agents playing.** ⬇️

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit10/soccertwos.gif" alt="Thumbnail"/>

## Reward Function

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit10/soccerreward.png" alt='reward-function' />

## Observation Space

The observation space is composed of vectors of size 336:

* 11 ray-casts forward distributed over 120 degrees (264 state dimensions)
* 3 ray-casts backward distributed over 90 degrees (72 state dimensions)
* Both of these ray-casts can detect 6 objects:
    * Ball
    * Blue Goal
    * Purple Goal
    * Wall
    * Blue Agent
    * Purple Agent

## Action Space

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit10/socceraction.png" alt="action-space" />

# Step 2: Understand MA-POCA

We know how to train agents to play against others: we can use self-play. This is a perfect technique for a 1vs1.

But in our case we’re 2vs2, and each team has 2 agents. How then can we train cooperative behavior for groups of agents?

As explained in the [Unity Blog](https://blog.unity.com/technology/ml-agents-v20-release-now-supports-training-complex-cooperative-behaviors), agents typically receive a reward as a group (+1 - penalty) when the team scores a goal. This implies that every agent on the team is rewarded even if each agent didn’t contribute the same to the win, which makes it difficult to learn what to do independently.

The Unity MLAgents team developed the solution in a new multi-agent trainer called **MA-POCA (Multi-Agent POsthumous Credit Assignment)**.

The idea is simple but powerful: a centralized critic processes the states of all agents in the team to estimate how well each agent is doing. Think of this critic as a coach.

This allows each agent to make decisions based only on what it perceives locally, and simultaneously evaluate how good its behavior is in the context of the whole group.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit10/mapoca.png" alt="MA_POCA"/>

The solution then is to use Self-Play with an MA-POCA trainer (called poca). The poca trainer will help us to train cooperative behavior and self-play to win against an opponent team. Self-play with config params described [here](https://huggingface.co/learn/deep-rl-course/unit7/self-play)

If you want to dive deeper into this MA-POCA algorithm, you need to read the paper they published [here](https://arxiv.org/pdf/2111.05992.pdf) and the sources we put on the additional readings section.

# Step 3: Define Config File

Now, you define the training hyperparameters in config.yaml files.

There are multiple hyperparameters. To understand them better, you should read the explanations for each of them in the documentation

The config file we’re going to use here is in ***./config/poca/SoccerTwos.yaml***.

Compared to Pyramids or SnowballTarget, we have new hyperparameters with a self-play part. How you modify them can be critical in getting good results.

The advice I can give you here is to check the explanation and recommended value for each parameters (especially self-play ones) against the [documentation](https://github.com/Unity-Technologies/ml-agents/blob/release_20_docs/docs/Training-Configuration-File.md).

Now that you’ve modified our config file, you’re ready to train your agents.

# Step 4: Train Agent

To train the agents, we need to launch mlagents-learn and select the executable containing the environment.

We define *four* parameters:

* mlagents-learn <config>: the path where the hyperparameter config file is.
* -env: where the environment executable is.
* -run_id: the name you want to give to your training run id.
* -no-graphics: to not launch the visualization during the training.

Depending on your hardware, 5M timesteps (the recommended value, but you can also try 10M) will take 5 to 8 hours of training. You can continue using your computer in the meantime, but I advise deactivating the computer standby mode to prevent the training from being stopped.

Depending on the executable you use (windows, ubuntu, mac) the training command will look like this (your executable path can be different so don’t hesitate to check before running).

The executable contains 8 copies of SoccerTwos.

⚠️ It’s normal if you don’t see a big increase of ELO score (and even a decrease below 1200) before 2M timesteps, since your agents will spend most of their time moving randomly on the field before being able to goal.

⚠️ You can stop the training with Ctrl + C but beware of typing this command only once to stop the training since MLAgents needs to generate a final .onnx file before closing the run.

In [ ]:
!mlagents-learn "./ml-agents/config/poca/SoccerTwos.yaml" --env="./ml-agents/training-envs-executables/SoccerTwos.exe" --run-id="SoccerTwos" --no-graphics --resume

## Resume Training With Modified Config

After you have modified the config or for some reasons, previous run terminated or stopped or you want to make your agent stronger. just add *--resume* at the end to continue training from the last saved agent

# Step 5: Push the agent to the Hugging Face Hub

Now that we trained our agents, we’re ready to push them to the Hub to be able to participate in the AI vs. AI challenge and visualize them playing on your browser🔥.

To be able to share your model with the community, there are three more steps to follow:

1️⃣ (If it’s not already done) create an account to HF ➡ https://huggingface.co/join

2️⃣ Sign in and store your authentication token from the Hugging Face website.

Create a new token (https://huggingface.co/settings/tokens) with write role

Copy the token, run this, and paste the token


In [ ]:
!huggingface-cli login

Then, we need to run mlagents-push-to-hf.

And we define *four* parameters:

* -run-id: the name of the training run id.
* -local-dir: where the agent was saved, it’s *results/<run_id name>*, so in my case results/First Training.
* -repo-id: the name of the Hugging Face repo you want to create or update. It’s always *<huggingface username / repo name>* If the repo does not exist it will be created automatically
* --commit-message: since HF repos are git repositories you need to give a commit message.

In [ ]:
!mlagents-push-to-hf  --run-id="SoccerTwos" --local-dir="./results/SoccerTwos" --repo-id="hishamcse/poca-SoccerTwos" --commit-message="First Push"

# Step 6: Verify that your model is ready for AI vs AI Challenge

Now that your model is pushed to the Hub, it’s going to be added automatically to the AI vs AI Challenge model pool. It can take a little bit of time before your model is added to the leaderboard given we do a run of matches every 4h.

But to ensure that everything works perfectly you need to check:

* That you have this tag in your model: *ML-Agents-SoccerTwos*. This is the tag we use to select models to be added to the challenge pool. To do that go to your model and check the tags
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit10/verify1.png" alt="tag" />

If it’s not the case you just need to modify the readme and add it
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit10/verify2.png" alt="readme" />

* That you have a SoccerTwos.onnx file

We strongly suggest that you create a new model when you push to the Hub if you want to train it again or train a new version.

# Step 7: Visualize some match in our demo

Now that your model is part of AI vs AI Challenge, you can visualize how good it is compared to others: https://huggingface.co/spaces/unity/ML-Agents-SoccerTwos

In order to do that, you just need to go to this demo:

* Select your model as team blue (or team purple if you prefer) and another model to compete against. The best opponents to compare your model to are either whoever is on top of the leaderboard or the [baseline](https://huggingface.co/unity/MLAgents-SoccerTwos) model

The matches you see live are not used in the calculation of your result but they are a good way to visualize how good your agent is.

And don’t hesitate to share the best score your agent gets on discord in the #rl-i-made-this channel